#  ETL ( Extract, Transform, Load)
ETL generically refers to the process of taking some data from source systems, (such as your CRM, ERP, or Marketing stack) transforming said data, and then loading it to some end system / application. 

**Extract**

Taking data from a source system
- Building pipelines to pull data from some source system(s) 
- Considerations around volume, speed (performance), cost

**Transform**

Manipulating Data for some end purpose, common transformations can include:
- Aggregation 
- Applying logic or calculations
- Joins, sorts, or pivots
- Applying data validation

**Load**

Loading or sending the data to some end system or application
- Output to a data warehouse
- Generating  flat files
- Updating / Creating tables in a database 

ETL is typically the domain of a data engineer, since there are typically considerations around volume, speed, and cost of shuttling data around a large enterprise systems. However, today you are typically seeing the rise of various developer positions that typically deal with creating automation around smaller volumes of data / non-critical systems.


![ETL](https://www.altexsoft.com/media/2019/06/word-image-29.png)

# Before Connecting to the DB
### Let's talk a little a bit about credentials in scripts

Items to be aware of with data connections in scripts:

- Do not store plain text passwords in scripts!
- Preferablly there should be some sort of encryption on credentials as you pass them
- Its generally a good practice to keep credentialing values in a seperate file to be referenced by your scripts


### How do we do this?

Network security, encryption, and best security practices are a entire domain within computer science. We will not be able to cover every aspect of it that you may need to know to optimally protect your systems or connections.

What I'll attempt to do here is show you a few techniques that you can use to make yourself more defensible and keep your credential information from being stored directly in your scripts.

# Building a JSON to push and pull credential information

In [ ]:
#Creating some token to reuse in scripts so you don't need to store credentials / sensitive info in the script
import json 

#example token created with a dictionary
example_token = {'key1':'value1','key2':'value2','key3':'value3'}

#saving that token(dictionary) as a json file
example = json.dumps(example_token)
# f = open("example_token.json","w")
# f.write(example)
# f.close()

In [ ]:
#loading a json token back into memory
with open('example_token.json', "r") as file:
    token = json.load(file)

#call the value of a particular key
print(token['key1'])

# Lets Build our Credential

What do we need in order to establish a connection to a MS based DB?

- Host name (Name of the machine the DB is on)
- User name 
- Password (***NOT*** *IN PLAIN TEXT*)
- Database

So how do we get our password into the credential without adding it in plain text?

ENTER - Cryptography!

In [4]:
from cryptography.fernet import Fernet

#generate a key that will allow you to encrypt / decrypt your password
key = Fernet.generate_key()

print(f'Your key is: {key}') # we'll store this in a seperate key file to decrypt our password

#create a cipher suite that will allow for encryption

#prefix string with b to pass it as bytecode
pword = b'Butthead1234' 

cipher_suite = Fernet(key) # used to encrypt / decrypt the password

ciphered_text = cipher_suite.encrypt(pword) #we'll store this in our credential file
print('')
print(f'Your encrypted password: {ciphered_text}')


#decrypt password 
unciphered_text = cipher_suite.decrypt(ciphered_text)
print('')
print(f'You password after decription: {unciphered_text}')

Your key is: b'QY7BJQg11ZmLklSCtPFJnYB6MZXRxrmpOCW9m47fniY='

Your encrypted password: b'gAAAAABe0vqYoS8CWAWNAE0-RVY8CCM2ub8qA62JqBSyaYUUNKIAhFAARSRvbOiGO6w29Iy7EAo_WVCC6OHP6WHwv3cI-5p8pg=='

You password after decription: b'Butthead1234'


In [ ]:
#lets create our credential to connect

#example token created with a dictionary
sql_token = {'server':r'py-please.database.windows.net',
             'user':r'py-please',
             'password':r'THIS IS NOT THE PASSWORD', # put the ciphered password here
             'database':r'py_please'}

#saving that token(dictionary) as a json file
sql = json.dumps(sql_token)
f = open("sql_token.json","w")
f.write(sql)
f.close()

#do the same thing to create a key file!

# SQL ALCHEMY / PYODBC



In [35]:
import json
#loading a json token back into memory
with open(r'../../sql_token_real.json', "r") as file:
    token = json.load(file)
    
with open(r'../../sql_key_real.json','r') as file:
    key = json.load(file)

In [36]:
import pyodbc
import pandas as pd

odbc driver
https://go.microsoft.com/fwlink/?linkid=2120137

In [41]:

server = token['server']
database = token['database']
username = token['user']
password = cipher_suite.decrypt(str.encode(token['password'])).decode()
driver= '{ODBC Driver 17 for SQL Server}'
cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)

stmt = r"""SELECT TOP(100) * FROM Owners"""

# Tables available for query:
# Owners
# Pets
# Procedures_Details
# Procedures_Histtory

df=pd.read_sql_query(stmt,cnxn)


In [42]:
df.head()

,OwnerID,Name,Surname,StreetAddress,City,State,StateFull,ZipCode
0,1070,Jessica,Velazquez,3861 Woodbridge Lane,Southfield,MI,Michigan,48034
1,1132,Rosa,Quarles,4791 Tennessee Avenue,Southfield,MI,Michigan,48034
2,1202,Susan,Jackson,3677 Daylene Drive,Livonia,MI,Michigan,48154
3,1306,Benjamin,Spears,1507 Twin Oaks Drive,Clam River,MI,Michigan,49612
4,1312,Charles,Chidester,4086 Cottonwood Lane,Dutton,MI,Michigan,49316


In [ ]:
#Output our SQL query data pull
df.to_csv('cad_logging.csv')